# Create product embedding for instacart dataset

Idea: similar to word2vec, products in the same order are close to each other

## Load orders and product name

In [15]:
!ls data

aisles.csv		       order_products__train.csv.zip
aisles.csv.zip		       orders.csv
departments.csv.zip	       orders.csv.zip
__MACOSX		       products.csv
order_products__prior.csv      products.csv.zip
order_products__prior.csv.zip  sample_submission.csv
order_products__train.csv      sample_submission.csv.zip


In [89]:
import pandas as pd

product_df = pd.read_csv("data/products.csv")
order_df = pd.read_csv("data/orders.csv")
order_prior_df = pd.read_csv("data/order_products__prior.csv")
# print(product_df.head(10))
# print(order_df.head(10))
# print(order_prior_df.head(10))



In [90]:
print(f"Number of products: {len(product_df)}")
print(f"Number of orders: {len(order_prior_df['order_id'].value_counts())}")

Number of products: 49688
Number of orders: 3214874


In [100]:
# Filter our order with less than 2 products
df1 = order_prior_df[order_prior_df.groupby("order_id")['order_id'].transform('size') > 20]
print(f"Number of orders with at least 2 products: {len(df1['order_id'].value_counts())}")

# Get list of products in these orders
product_ids = df1["product_id"].unique().tolist()
product_df1 = product_df[product_df["product_id"].isin(product_ids)]
print(f"Number of reduced products: {len(product_ids)}")

Number of orders with at least 2 products: 303410
Number of reduced products: 46829


In [101]:
# Build product mapping

product_ids = product_df1["product_id"]
product_names = product_df1["product_name"]
# map product id to indices starting from 0
product_id_to_ind = {product_id: i for i, product_id in enumerate(product_ids)}
product_ind_to_name = {i: product_name for i, product_name in enumerate(product_names)}
product_name_to_ind = {product_name: i for i, product_name in enumerate(product_names)}

In [102]:
import tqdm
def build_data(order_df, product_id_to_ind, window=2):
    unique_orders = order_df["order_id"].unique().tolist()
    order_to_ind = {
        order_id: i
        for i, order_id in enumerate(unique_orders)
    }
    # self.orders to store a list of orders, each order is represented by a list of products in that order
    orders = [[] for _ in unique_orders]
    for order_id, product_id in tqdm.tqdm(zip(order_df["order_id"], order_df["product_id"])):
        order_ind = order_to_ind[order_id]
        product_ind = product_id_to_ind[product_id]
        orders[order_ind].append(product_ind)
#     num_products = len(product_id_to_ind)        
    data = []
    for products in tqdm.tqdm(orders):
        for i, product in enumerate(products):
            start = max(0, i - window)
            end = min(len(products), i + window + 1)
            data.extend([
                (product, products[j])
                for j in range(start, end) if j !=i
            ])
    return data

data = build_data(df1, product_id_to_ind, window=2)

8215460it [00:02, 3120760.28it/s]
100%|██████████| 303410/303410 [00:06<00:00, 48032.01it/s]


In [103]:
# torch dataset
import torch
import random
from torch import nn
from torch.utils.data import DataLoader, Dataset
import tqdm

class ProductPairDataset(Dataset):
    """Dataset class that returns a pair of (context, target) product ids.
    
    The pair is a random combination of 2 products in the same order.
    
    """
    def __init__(self, data, num_products, num_noisy_products: int = 200, window=2):
        
        
        self.num_noisy_products = num_noisy_products
        self.num_products = num_products
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]
        
training_data = ProductPairDataset(data, num_products=len(product_id_to_ind))
  

In [104]:
len(training_data)

31041380

In [105]:

# batch_size = 8
# train_dataloader = DataLoader(
#     training_data, batch_size=batch_size, shuffle=True, num_workers=0
# )
import time

for i in range(5):
    batch_size = 2**(i+1)
    train_dataloader = DataLoader(
        training_data, batch_size=batch_size, shuffle=False, num_workers=0
    )
    t0 = time.time()
    count = 0
    for batch in train_dataloader:
#         print(batch)
        count += 1
        if count == 2:
            break
    
    print(f"batch_size = {batch_size}: execution time {time.time() - t0}")

batch_size = 2: execution time 0.0004963874816894531
batch_size = 4: execution time 0.00023746490478515625
batch_size = 8: execution time 0.0001633167266845703
batch_size = 16: execution time 0.0001659393310546875
batch_size = 32: execution time 0.0001785755157470703


In [106]:
data[1]


(12417, 44599)

In [107]:
# Build the pytorch lightning model
import pytorch_lightning as pl
import torch.multiprocessing
import torch.nn.functional as F
class SigmoidBCELoss(nn.Module):
    "BCEWithLogitLoss with masking on call."

    def __init__(self):
        super().__init__()

    def forward(self, inputs, target, mask=None):
#         breakpoint()
        out = nn.functional.binary_cross_entropy_with_logits(
            inputs, target, weight=mask, reduction="mean")
#         breakpoint()
        return out #.mean(dim=1)

loss_fn = SigmoidBCELoss()
loss2 = torch.nn.CrossEntropyLoss()

class Prod2VecModel(pl.LightningModule):
    def __init__(self, num_products, embed_size: int = 50):
        super().__init__()
        self.embed_size = embed_size
        self.embedding = nn.Embedding(num_products, self.embed_size)
        self.hidden = nn.Linear(self.embed_size, num_products, bias=True)


    def forward(self, contexts):
#         breakpoint()
        hid = self.embedding(contexts)
        pre_sigmoid = self.hidden(hid)
        return pre_sigmoid

    def training_step(self, batch, batch_idx):
#         contexts, targets, masks = batch
#         output = self.forward(contexts)
#         loss = loss_fn(output, targets, masks)
#         breakpoint()
#         loss = loss2(output, targets)

        contexts, targets = batch
        output = self.forward(contexts)
        loss = loss2(output, targets)
#         breakpoint()
#         loss = loss_fn(output, targets, None)

        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(), lr=10, momentum=0.1, weight_decay=1e-3
        )  # learning rate
        return optimizer

In [ ]:
num_products = len(product_id_to_ind)
embed_size = 20
model = Prod2VecModel(num_products, embed_size)
trainer = pl.Trainer(gpus=1, max_epochs=10)
trainer.fit(model, train_dataloader, train_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/home/tiepvu/w/tabml/tabml_env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 936 K 
1 | hidden    | Linear    | 983 K 
----------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.680     Total estimated model params size (MB)
/home/tiepvu/w/tabml/tabml_env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoad

Training: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [73]:
torch.save(model.state_dict(), 'model.pt')

In [74]:
# Model class must be defined somewhere
model = torch.load('model.pt')
# model.eval()

In [75]:
model

OrderedDict([('embedding.weight',
              tensor([[ 0.7804, -0.3046, -0.2718,  ..., -0.0812,  0.2188, -0.8614],
                      [-0.0475,  0.0458,  0.0286,  ...,  0.0018, -0.0108,  0.0400],
                      [-0.0334, -0.0475, -0.0121,  ...,  0.0027, -0.0219,  0.0674],
                      ...,
                      [-0.0135, -0.0036,  0.0067,  ...,  0.0032, -0.0077,  0.0243],
                      [-0.0895,  0.0473,  0.0071,  ..., -0.0022, -0.0295,  0.0533],
                      [-0.0286, -0.0014,  0.0095,  ...,  0.0046, -0.0155,  0.0409]])),
             ('hidden.weight',
              tensor([[ 0.7914, -0.3303, -0.2914,  ..., -0.0881,  0.2216, -0.8666],
                      [-0.0629,  0.0386,  0.0343,  ...,  0.0052, -0.0199,  0.0694],
                      [-0.0393, -0.0512, -0.0089,  ...,  0.0045, -0.0251,  0.0810],
                      ...,
                      [-0.0323, -0.0080,  0.0152,  ...,  0.0074, -0.0180,  0.0572],
                      [-0.1011,  0.044

In [76]:
embs = model['embedding.weight']

In [77]:
embs_arr = embs.detach().numpy()

In [78]:
embs_arr.shape

(1308, 20)

In [79]:
# write a class to find nearest neighbors

product_ids.iloc[0]

25

In [80]:
from tabml.utils import embedding

In [81]:
product_ids.iloc[1]

34

In [82]:
product_df

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [83]:
product_names

24         Salted Caramel Lean Protein & Fiber Bar
33                            Peanut Butter Cereal
44                               European Cucumber
92                     Uncured Cracked Pepper Beef
209      Homemade Hot Arrabbiata Fra Diavolo Sauce
                           ...                    
49325                      Cereal Variety Fun Pack
49394                  Geranium Scented Soy Candle
49546           Kids Defense Spray Sunscreen SPF50
49627                            Yoghurt Blueberry
49682                               Cucumber Kirby
Name: product_name, Length: 1308, dtype: object

In [84]:
names = product_names.tolist()

In [88]:
def find_similar(embs_arr, ind, names):
    ids = embedding.find_nearest_neighbors(embs_arr[ind], embs_arr, measure="dot", k=4)
    return [names[ind] for ind in ids]

def find_similar_by_name(embs_arr, sub_name, names):
    ids = [ind for ind in range(len(names)) if sub_name in names[ind]]
    for ind in ids:
        print('==========')
        print(f'Similar items of "{names[ind]}":')
        print(find_similar(embs_arr, ind, names))
        
find_similar_by_name(embs_arr, 'Hazelnuts', names)

Similar items of "Dark Chocolate with Whole Hazelnuts":
['Dark Chocolate with Whole Hazelnuts', 'Muffins, Mini, Flax, Chocolate Brownie', 'Pizza Poppers, Three Cheese', 'Almond Milk Peach Yogurt', 'Frosted berry Strawberry Toaster Pastries', 'Toasted Caramel Coconut Chips', 'Enchilada Black Bean Vegetable', 'Green Supreme Apple, Kale & Lemon Cold Pressured Fruit & Vegetable Juice', 'Haunted Ghost Pepper Chips', 'Original Veggie Straws']
